In [4]:
import pandas as pd
import spacy
import pickle
import time
import random

In [6]:
train_data = pickle.load(open("main_training_set","rb"))
def reformat_tagged_data(data):
    TRAIN_DATA = []
    for item in train_data:
        new_data = (item[0],{"entities":item[1]})
        TRAIN_DATA.append(new_data)
    # Now we have reformatted TRAIN_DATA ready for model input
    return TRAIN_DATA
TRAIN_DATA = reformat_tagged_data(train_data)
print(TRAIN_DATA[:1])

[('In 1947 Portland, Maine, banker Andy Dufresne is convicted of murdering his wife and her lover and is sentenced to two consecutive life sentences at the Shawshank State Penitentiary', {'entities': [(32, 45, 'PERSON')]})]


In [3]:
nlp = spacy.load("en_core_web_sm")
optimizer = nlp.begin_training()
for i in range(20):
    random.shuffle(TRAIN_DATA)
    for text, annotations in TRAIN_DATA:
        nlp.update([text], [annotations], drop=0.5, sgd=optimizer)
nlp.to_disk("/model")

C:\Users\jmarchant\AppData\Local\Continuum\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Despite Morpheus's attempt to guide Neo to safety ..." with entities "[(65, 68, 'PERSON'), (110, 115, 'PERSON'), (124, 1...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\jmarchant\AppData\Local\Continuum\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Barbossa takes her prisoner, believing her to be t..." with entities "[(65, 69, 'PERSON'), (90, 96, 'PERSON'), (0, 8, 'P...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\jmarchant\AppData\Local\Continuum\anaconda3

In [1]:
def test_model(nlp,TEST_DATA):
    total = 0
    correct = 0
    completely_correct = 0
    for example in TEST_DATA:
        doc = nlp(example[0])
        
        actual_positions = [(x[0],x[1]) for x in example[1]["entities"]]
        predictions = [str(x) for x in list(doc.ents)]
        actual_characters = [str(doc)[a:b] for (a,b) in actual_positions]
        initial_predictions = predictions
        
        for char in actual_characters:
            if char in predictions:
                correct += 1
                predictions.remove(char)
            total += 1
        
        if len(predictions) == 0:
            completely_correct += 1
        
#         print("Sentence: {}".format(doc))
#         print("Predictions: {}".format(initial_predictions))
#         print("Actual: {}".format(actual_characters))
#         print()
    print("{}% characters correctly tagged".format(round(100*(correct/total),0)))
    print("{}% sentences perfectly tagged".format(round(100*(completely_correct/len(TEST_DATA)),0)))

In [7]:
nlp = spacy.load("/model")
test_model(nlp,TRAIN_DATA)

92.0% characters correctly tagged
96.0% sentences perfectly tagged
